# Tratamiento del fichero csv de cervezas obtenido de la web *soloartesanas.es*

In [1]:
import numpy as np
import pandas as pd
import yaml

#pd.options.display.max_colwidth = 300

In [2]:
# Importo el fichero "soloartesanas.csv"

In [3]:
df_soloartesanas=pd.read_csv("data/df_soloartesanas.csv", sep=";")
df_soloartesanas.head()

,Unnamed: 0,id,color,name,brand,rate_beer,features,description,image
0,0,1,rubia,"Arriaca Radler, cerveza artesana con limón (lata)",(cervezas artesanas Cervezas Arriaca),NaN,"{'Estilo de cerveza': ' Blonde Ale', 'Graduaci...",\nLa primera cerveza Radler artesana creada en...,https://static2.soloartesanas.es/3437-large_de...
1,1,2,rubia,Alegría & Boris Brew Ipanosuarus Rex,(cervezas artesanas Cervezas Alegría),NaN,{'Estilo de cerveza': ' Imperial / Double IPA'...,"\nUna doble IPA rubia, potente y seca. Elabora...",https://static3.soloartesanas.es/3333-large_de...
2,2,3,rubia,Roy The Bull Mango,(cervezas artesanas Roy The Bull),NaN,"{'Estilo de cerveza': ' Fruit Beer', 'Graduaci...",\nAquí está la primera cerveza de Roy The Bull...,https://static3.soloartesanas.es/3332-large_de...
3,3,4,rubia,Marijuana,(cervezas artesanas Marijuana),NaN,"{'Estilo de cerveza': ' Pale Ale', 'Graduación...",\nLa única cerveza cerveza artesana que podrás...,https://static1.soloartesanas.es/3223-large_de...
4,4,5,rubia,Arriaca Trigo (lata),(cervezas artesanas Cervezas Arriaca),NaN,"{'Estilo de cerveza': ' Weizen - Weissbier', '...",\nUna cerveza de trigo muy afrutada y ligera. ...,https://static2.soloartesanas.es/3449-large_de...


In [4]:
df_soloartesanas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   435 non-null    int64 
 1   id           435 non-null    int64 
 2   color        435 non-null    object
 3   name         435 non-null    object
 4   brand        435 non-null    object
 5   rate_beer    157 non-null    object
 6   features     435 non-null    object
 7   description  433 non-null    object
 8   image        435 non-null    object
dtypes: int64(2), object(7)
memory usage: 30.7+ KB


In [5]:
# Importamos un fichero con cervezas artesanas internacionales por si nos sirve para completar
# el que usaremos en el estudio
df_aux=pd.read_csv("data/Labirratorum.csv", sep=",")
df_aux.head()

,Unnamed: 0,id,name,price,descr_short,descr_full,image,brand,ean,Estilo,...,Porcentaje_Alcohol,Volumen_cl,Tipo Fermentación,Maltas,Lúpulos,IBU,Color,Envase,Otros ingredientes,Levadura
0,0,lb_1,Anchor Porter,2.90,Porter,"Cerveza negra, cremosa, con aromas a chocolate...",https://www.labirratorium.com/41-large_default...,Anchor Brewing Company (Sapporo),7.278300e+10,AMERICAN PORTER,...,"5,6",35.5,Ale,"2-Row Pale. Caramelo, Chocolate, Black",Northern Brewer,20.0,Negro,Botella,NaN,NaN
1,1,lb_2,Störtebeker Schwarz-Bier,2.40,Cerveza negra de baja fermentación,Cerveza negra de baja fermentación de estilo S...,https://www.labirratorium.com/488-large_defaul...,Störtebeker,4.014807e+12,SCHWARZBIER,...,5,50,Lager (Baja Fermentación),Cebada,NaN,20.0,Negro,Botella,NaN,NaN
2,2,lb_3,Weihenstephaner Vitus,2.65,Weizenbock,Cerveza refrescante de trigo fuerte y aromátic...,https://www.labirratorium.com/43-large_default...,Weihenstephan,4.105120e+12,TRIGO ALEMÁN (WEIZENDOPPELBOCK),...,"7,7",50,Ale (Alta Fermentación),Trigo y Cebada,NaN,20.0,Amarillo dorado,Botella,NaN,NaN
3,3,lb_4,Bavaria Wit 0.0%,1.60,"Cerveza holandesa de trigo sin alcohol, de col...","Cerveza holandesa sin alcohol, de color amaril...",https://www.labirratorium.com/53-large_default...,Bavaria,NaN,SIN ALCOHOL,...,0.0,33,Ale (Alta Fermentación),NaN,NaN,20.0,Amarillo pálido,Botella,NaN,NaN
4,4,lb_5,Laugar EPA! Euskadiko Pale Ale,2.55,American Pale Ale,American Pale Ale de 5.2% ABV elaborada con lú...,https://www.labirratorium.com/11103-large_defa...,Laugar,NaN,APA (AMERICAN PALE ALE),...,5.4,33,Ale (Alta Fermentación),Cebada,"Cascade, Chinook y Columbus",40.0,Amarillo dorado,Botella,NaN,NaN


## Limpieza inicial del fichero
Revisión inicial del dataframe para eliminar columnas inncecesarias, agrupar información, unificar tipos y contenidos, etc. de modo que se pueda realizar después un análisis de la información.

Para empezar, gran parte de la información está recogida en el campo "features" que tiene estructura de diccionario y al que habrá que extraer la información separándola en columnas.

In [6]:
# Veo cómo es el campo "features" con un ejemplo
df_soloartesanas.iloc[355, :]['features']

"{'Estilo de cerveza': ' Stout', 'Graduación alcohólica': ' 4,2% Alc./Vol.', 'Amargor (según la Unidad Internacional de Amargor)': ' 40 IBUs', 'Color': 'Negra', 'Grano': 'Avena', 'Fermentación': 'Ale', 'European Brewing Convention (EBC)': '110', 'Maltas': 'Maris Otter Floor Malt, Chocolate, Roasted Barley, Crystal y Avena', 'Lúpulos': 'Cascade', 'Población': 'Liérganes', 'Provincia': 'Cantabria', 'Comunidad Autónoma': 'Cantabria', 'País': 'España'}"

In [7]:
# para ver las características del registro más largo

df_soloartesanas["num_caract"]= df_soloartesanas['features'].apply(len)

features_mas_largo=df_soloartesanas[df_soloartesanas["num_caract"]==df_soloartesanas["num_caract"].max()]['features']
df_soloartesanas.drop(columns="num_caract",inplace=True)

features_mas_largo.values

array(["{'Estilo de cerveza': ' Amber Ale', 'Graduación alcohólica': ' 5.3% Alc./Vol.', 'Amargor (según la Unidad Internacional de Amargor)': ' 30 IBUs', 'Color': 'Tostada', 'Grano': 'Cebada', 'Fermentación': 'Ale', 'European Brewing Convention (EBC)': '40', 'Maltas': 'Pale Ale, Crystal, Victoria y CaraAroma', 'Lúpulos': 'Kent Golding y Fuggles', 'Población': 'Oviedo', 'Provincia': 'Asturias', 'Comunidad Autónoma': 'Asturias', 'País': 'España', 'Temperatura de consumo': '10-13ºC', 'Vaso recomendado': 'Vaso ancho'}"],
      dtype=object)

In [8]:
# Hay que transformar la columna de features, que es un diccionario, en varias columnas 
df_soloartesanas['features']

0      {'Estilo de cerveza': ' Blonde Ale', 'Graduaci...
1      {'Estilo de cerveza': ' Imperial / Double IPA'...
2      {'Estilo de cerveza': ' Fruit Beer', 'Graduaci...
3      {'Estilo de cerveza': ' Pale Ale', 'Graduación...
4      {'Estilo de cerveza': ' Weizen - Weissbier', '...
                             ...                        
430    {'Estilo de cerveza': ' English Pale Ale', 'Gr...
431    {'Estilo de cerveza': ' Red Ale', 'Graduación ...
432    {'Estilo de cerveza': ' Irish Red Ale', 'Gradu...
433    {'Estilo de cerveza': ' Red Ale', 'Graduación ...
434    {'Estilo de cerveza': ' Red Ale', 'Graduación ...
Name: features, Length: 435, dtype: object

In [9]:
# aplicar yaml a columna features y convertirla en diccionario aplicando map
# Para un registro sería: mi_dict2 = yaml.load(df_soloartesanas.features[0])

df_soloartesanas['features'] = df_soloartesanas.features.map(lambda x: yaml.load(x))

<ipython-input-9-f3f490e69a13>:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  df_soloartesanas['features'] = df_soloartesanas.features.map(lambda x: yaml.load(x))


In [10]:
df_soloartesanas['features']

0      {'Estilo de cerveza': ' Blonde Ale', 'Graduaci...
1      {'Estilo de cerveza': ' Imperial / Double IPA'...
2      {'Estilo de cerveza': ' Fruit Beer', 'Graduaci...
3      {'Estilo de cerveza': ' Pale Ale', 'Graduación...
4      {'Estilo de cerveza': ' Weizen - Weissbier', '...
                             ...                        
430    {'Estilo de cerveza': ' English Pale Ale', 'Gr...
431    {'Estilo de cerveza': ' Red Ale', 'Graduación ...
432    {'Estilo de cerveza': ' Irish Red Ale', 'Gradu...
433    {'Estilo de cerveza': ' Red Ale', 'Graduación ...
434    {'Estilo de cerveza': ' Red Ale', 'Graduación ...
Name: features, Length: 435, dtype: object

In [11]:
# veo un ejemplo
print(df_soloartesanas.iloc[355, :])

Unnamed: 0                                                   355
id                                                           356
color                                                      negra
name                                     Dougall's Session Stout
brand                             (cervezas artesanas Dougall's)
rate_beer                                               92 / 100
features       {'Estilo de cerveza': ' Stout', 'Graduación al...
description    \nDougalls Session Stout es una excelente cerv...
image          https://static3.soloartesanas.es/2095-large_de...
Name: 355, dtype: object


In [12]:
# Separo en columnas cada clave/valor del dicc de features

df_soloartesanas = pd.concat([df_soloartesanas.drop(['features'], axis=1), df_soloartesanas['features'].apply(pd.Series)], axis=1)

In [13]:
df_soloartesanas.head(2)

,Unnamed: 0,id,color,name,brand,rate_beer,description,image,Estilo de cerveza,Graduación alcohólica,...,Comunidad Autónoma,Lúpulos,European Brewing Convention (EBC),Maltas,Temperatura de consumo,Vaso recomendado,Grano,Sin gluten,Ecológica,País
0,0,1,rubia,"Arriaca Radler, cerveza artesana con limón (lata)",(cervezas artesanas Cervezas Arriaca),NaN,\nLa primera cerveza Radler artesana creada en...,https://static2.soloartesanas.es/3437-large_de...,Blonde Ale,"2,7% Alc./Vol.",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,rubia,Alegría & Boris Brew Ipanosuarus Rex,(cervezas artesanas Cervezas Alegría),NaN,"\nUna doble IPA rubia, potente y seca. Elabora...",https://static3.soloartesanas.es/3333-large_de...,Imperial / Double IPA,"8,2% Alc./Vol.",...,Valencia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Compruebo las nuevas columnas de features, tus tipos y misings
df_soloartesanas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 25 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   Unnamed: 0                                          435 non-null    int64 
 1   id                                                  435 non-null    int64 
 2   color                                               435 non-null    object
 3   name                                                435 non-null    object
 4   brand                                               435 non-null    object
 5   rate_beer                                           157 non-null    object
 6   description                                         433 non-null    object
 7   image                                               435 non-null    object
 8   Estilo de cerveza                                   419 non-null    object
 9   Graduación

In [15]:
# Algunas de estas nuevas columnas las podré borrar si no aportan información y no están en los otros ficheros
# Otras las puedo unir en una única columna, como el origen
# Algunas características que no estén completas, si aparecen en otros ficheros, las dejaré con sus NAN
# En la app podrán seleccionarse los campos a desplegar en función de si hay dato

In [16]:
'''
RangeIndex: 435 entries, 0 to 434
Data columns (total 25 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   Unnamed: 0                                          435 non-null    int64 -- Borrar
 1   id                                                  435 non-null    int64 
 2   color                                               435 non-null    object
 3   name                                                435 non-null    object
 4   brand                                               435 non-null    object
 5   rate_beer                                           157 non-null    object -- Borrar
 6   description                                         433 non-null    object -- completar
 7   image                                               435 non-null    object
 8   Estilo de cerveza                                   419 non-null    object
 9   Graduación alcohólica                               415 non-null    object
 10  Color                                               418 non-null    object -- Borrar (duplicada)
 11  Fermentación                                        399 non-null    object
 12  Amargor (según la Unidad Internacional de Amargor)  275 non-null    object
 13  Población                                           409 non-null    object -- Unificar en "Origen"
 14  Provincia                                           426 non-null    object -- Unificar en "Origen"
 15  Comunidad Autónoma                                  424 non-null    object -- Unificar en "Origen"
 16  Lúpulos                                             99 non-null     object
 17  European Brewing Convention (EBC)                   101 non-null    object
 18  Maltas                                              87 non-null     object
 19  Temperatura de consumo                              137 non-null    object
 20  Vaso recomendado                                    52 non-null     object -- Borrar/añadir a descr
 21  Grano                                               310 non-null    object
 22  Sin gluten                                          2 non-null      object -- Borrar
 23  Ecológica                                           3 non-null      object -- Borrar
 24  País                                                45 non-null     object -- Unificar en "Origen"
dtypes: int64(2), object(23)

'''
print(" ")

In [17]:
# compruebo si las cervezas sin gluten y las ecológicas tienen esa info en su descripción (se podrían borrar)
sin=df_soloartesanas[df_soloartesanas["Sin gluten"]=='No']["description"]
list(sin)

['\nUna cerveza artesana para disfrutar del buen tiempo y del calor que recorre nuestra geografía durante los meses cálidos. Los chicos de La Virgen elaboran La Virgen Veraniega para época\xa0estival con fecha límite de producción, Septiembre. Y lo hacen para ofrecerte una cerveza genuina y adaptada a las exigencias meteorológicas del calor.\nEs una cerveza de estilo Pale Ale que usa levadura de alta fermentación California Lager. Todo acompañado de una incipiente presencia de amargor con un gran equilibrio a malta, hace de ella una cerveza artesana muy refrescante.\nDisfruta y refréscate este verano\n']

In [18]:
sin=df_soloartesanas[df_soloartesanas["Ecológica"]=='Sí']["description"]
list(sin)

['\nLlega una nueva evolución de Armando... que ahora se convierte en "The Tree Man", una cerveza orgánica y ecológica (como toda artesana, sin aditivos ni conservantes ni colorantes ni acelerantes). Una cerveza 100% natural.\nDisfruta de esta cerveza de estilo Pale Ale de corte inglés, fresca y con aromas y matices a frutas tropicales.\n',
 '\nBeauy es una cerveza completa. Es una gran cerveza: es ecológica, es artesana y uno de sus principales componentes es el Aloe Vera.\nLas base en una Pale Ale de estilo americano, sobre la que se elabora esta cerveza vegetal suave, equilibrada y ligera. Gran presencia dorada en copa, de la que saldran los primeros aromas de miel y caramelo, y al instante llegará el herbal del aloe. En boca resulta poco amarga y muy fresca, creando una cerveza muy fácil de beber.\nDisfruta de la buena cerveza\n']

In [19]:
# borro las columnas que no aportan valor por estar repetidas o tener muy pocos datos: 
# "Unnamed: 0","Vaso recomendado","Sin gluten","Ecológica" y "Color"

df_soloartesanas.drop(columns=["Unnamed: 0","rate_beer","Color","Vaso recomendado","Sin gluten","Ecológica"], inplace=True)

In [20]:
df_soloartesanas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 19 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   id                                                  435 non-null    int64 
 1   color                                               435 non-null    object
 2   name                                                435 non-null    object
 3   brand                                               435 non-null    object
 4   description                                         433 non-null    object
 5   image                                               435 non-null    object
 6   Estilo de cerveza                                   419 non-null    object
 7   Graduación alcohólica                               415 non-null    object
 8   Fermentación                                        399 non-null    object
 9   Amargor (s

In [21]:
# veo qué cervezas están sin descripción
df_soloartesanas[df_soloartesanas["description"].isna()]

,id,color,name,brand,description,image,Estilo de cerveza,Graduación alcohólica,Fermentación,Amargor (según la Unidad Internacional de Amargor),Población,Provincia,Comunidad Autónoma,Lúpulos,European Brewing Convention (EBC),Maltas,Temperatura de consumo,Grano,País
171,172,rubia,Catalina,(cervezas artesanas Torquemada),NaN,https://static2.soloartesanas.es/886-large_def...,Pilsner,"4,8% Alc./Vol.",Ale,NaN,Torquemada,Palencia,Castilla y León,NaN,NaN,NaN,2-4ºC,Cebada,NaN
223,224,rubia,Illusió,(cervezas artesanas Ginesart),NaN,https://static2.soloartesanas.es/2664-large_de...,Iber Ale,"5,0% Alc./Vol.",Ale,NaN,Ginesart,Tarragona,Cataluña,Cascade y Saaz. Salvo,NaN,"Pale ale, Pilsen y de trigo",8-12 ºC,Trigo,NaN


In [22]:
# Añado una descripción de las cervezas que faltan y que encuentro en otras webs
df_soloartesanas.loc[171,"description"]="Se trata de una cerveza rubia tipo Pils, ligera y refrescante a base de malta Pilsner y lúpulos alemanes y checos. Catalina debe su nombre al nacimiento, el 14 de enero de 1507, de la Infanta Catalina de Austria (reina de Portugal) en la localidad Torquemada. Hija de Juana de Castilla y Felipe de Habsburgo. La cerveza Catalina es una cerveza rubia brillante, ligeramente amarga, con olores a cereal y una espuma equilibrada. El contenido alcohólico es de 4,8% Vol. lo que hace que esta cerveza sea la más ligera de toda la gama. Temperatura de consumo recomendada: 2º-4º"
df_soloartesanas.loc[223,"description"]="Estilo cervecero: Iber Ale. Cerveza rubia de alta fermentación adaptada al estilo de nuestro país. Nota de cata: Toda la intensidad del lúpulo en combinación con la malta, para hacer pasar la sed. Ingredientes: Malta Pale ale, Pilsen y de trigo. de los lúpulos: Cascade y Saaz. Levadura. Temperatura de consumo: entre 8 y 12 º C Graduación alcohólica: 5% Alc. Vol."

In [23]:
df_soloartesanas.iloc[223]["description"]

'Estilo cervecero: Iber Ale. Cerveza rubia de alta fermentación adaptada al estilo de nuestro país. Nota de cata: Toda la intensidad del lúpulo en combinación con la malta, para hacer pasar la sed. Ingredientes: Malta Pale ale, Pilsen y de trigo. de los lúpulos: Cascade y Saaz. Levadura. Temperatura de consumo: entre 8 y 12 º C Graduación alcohólica: 5% Alc. Vol.'

### Unifico la información sobre el origen de fabricación
Por la propia página web sabemos que todas las cervezas son de origen español, pero revisamos los datos para ver si falta información y si hay uniformidad en la nomenclaruta.

In [24]:
# compruebo que efectivamente solo hay cervezas españolas
print("Sin país: ", df_soloartesanas["País"].isna().sum())
df_soloartesanas.groupby("País")["id"].nunique()

Sin país:  390


País
ESpaña     1
España    44
Name: id, dtype: int64

Hay demasiadas cervezas sin identifiador de país. Puedo forzar la columna y poner en todas "España", pero antes voy a comprobar los otros campos sobre población, provincia y comunidad autónoma, para asegurarme de que no hay cervezas de otros.

Del total de 435 tenemos datos sobre:
- Población:                                           409 
- Provincia:                                           426 
- Comunidad Autónoma:                                 424 



In [25]:
# Compruebo las provincias que aparecen
df_soloartesanas["Provincia"].unique()

array([nan, 'Valencia', 'Pontevedra', 'Ávila', 'Guadalajara', 'Barcelona',
       'Segovia', 'Alicante', 'Burgos', 'La Rioja', 'Vizcaya', 'Madrid',
       'Córdoba', 'Málaga', 'Teruel', 'Valladolid', 'Badajoz',
       'Castellón', 'Cuenca', 'Tarragona', 'Girona', 'Álava', 'Albacete',
       'Soria', 'Lugo', 'Navarra', 'León', 'Palencia', 'Murcia', 'Huelva',
       'Asturias', 'Toledo', 'Cantabria', 'Sevilla', 'Ciudad Real',
       'Orense', 'La Coruña', 'Huesca', 'Lérida', 'Cádiz', 'Granada',
       'Guipúzcoa', 'Zaragoza', 'Zamora', 'Salamanca', 'Cáceres'],
      dtype=object)

In [26]:
# Estas son las CC.AA que aparecen
df_soloartesanas["Comunidad Autónoma"].unique()

array([nan, 'Valencia', 'Galicia', 'Castilla y León',
       'Castilla-La Mancha', 'Cataluña', 'La Rioja', 'País Vasco',
       'Madrid', 'Andalucía', 'Aragón', 'Extremadura', 'Navarra',
       'Murcia', 'Asturias', 'Canarias', 'Cantabria'], dtype=object)

In [27]:
# ¿Que CCAA faltan de las que sí sabemos la provincia?
df_soloartesanas[df_soloartesanas["Comunidad Autónoma"].isna()]["Provincia"]


0         NaN
29        NaN
30        NaN
41        NaN
63     Girona
69        NaN
230       NaN
351    Girona
352    Girona
372       NaN
396       NaN
Name: Provincia, dtype: object

In [28]:
# como son todas de Girona, actualizamos la CCAA con Cataluña
mask=(df_soloartesanas["Comunidad Autónoma"].isna()) & (df_soloartesanas["Provincia"]=="Girona")
df_soloartesanas.loc[mask,"Comunidad Autónoma"]="Cataluña"

In [29]:
# comprobamos que se han actualizado bien
df_soloartesanas[df_soloartesanas["Comunidad Autónoma"].isna()]["Provincia"]

0      NaN
29     NaN
30     NaN
41     NaN
69     NaN
230    NaN
372    NaN
396    NaN
Name: Provincia, dtype: object

In [30]:
# A la inversa, en las provincias faltan ¿cuál es la CC.AA.?
df_soloartesanas[df_soloartesanas["Provincia"].isna()]["Comunidad Autónoma"]

0           NaN
29          NaN
30          NaN
41          NaN
69          NaN
230         NaN
257    Cataluña
372         NaN
396         NaN
Name: Comunidad Autónoma, dtype: object

Comprobamos que las 8 provincias que faltan no tienen información sobre CC.AA, salvo 1 que es de Cataluña.
El siguiente paso será averiguar qué información tenemos y falta respecto a la población de origen, para ver si se pueden completar las otras columnas.

In [31]:
# Por un lado, en las provincias que faltan ¿cuál es población?
df_soloartesanas[df_soloartesanas["Provincia"].isna()]["Población"]

0             NaN
29            NaN
30            NaN
41            NaN
69            NaN
230           NaN
257    Granollers
372           NaN
396           NaN
Name: Población, dtype: object

In [32]:
# Lo mismo para las CC.AA.
df_soloartesanas[df_soloartesanas["Comunidad Autónoma"].isna()]["Población"]

0      NaN
29     NaN
30     NaN
41     NaN
69     NaN
230    NaN
372    NaN
396    NaN
Name: Población, dtype: object

Con la información de la columna "Población" solamente podemos completar los datos del registro 257 (Granollers), que corresponden a la provincia de Barcelona en Cataluña.

In [33]:
# Actualizo el registro 257 (Granollers)
df_soloartesanas.loc[257,"Provincia"]="Barcelona"

In [34]:
print("Cervezas con población de origen",df_soloartesanas["Población"].count())
print("Cervezas con provincia",df_soloartesanas["Provincia"].count())
print("Cervezas con CC.AA",df_soloartesanas["Comunidad Autónoma"].count())


Cervezas con población de origen 409
Cervezas con provincia 427
Cervezas con CC.AA 427


Esto significa que de las 435 cervezas recogidas 409 tienen toda la información, 8 no tienen ningún dato sobre procedencia y 18 tienen información regional.
Dada esta circunstancia, se decide:
1. Eliminar los 8 registros con origen totalmente desconocido.
2. Conservar los 18 incompletos, indicando que la población de origen está "sin especificar". 
3. Unificar como país España

Con ello, conseguiríamos un dataset con 427 registros documentados.

In [35]:
# Elimino los 8 registros sin ninguna información sobre su origien
mask=df_soloartesanas[df_soloartesanas["Comunidad Autónoma"].isna()]
df_soloartesanas=df_soloartesanas.drop(mask.index)


In [36]:
# ¿Cuáles son las poblaciones que faltan?
df_soloartesanas[df_soloartesanas["Población"].isna()]["Provincia"]

18        Valencia
27          Madrid
39      Valladolid
40      Valladolid
88         Navarra
94       Barcelona
97       Cantabria
172    Guadalajara
173    Guadalajara
263        Vizcaya
283         Madrid
347         Girona
353    Guadalajara
386    Guadalajara
417     Valladolid
419      Barcelona
421      Salamanca
428    Guadalajara
Name: Provincia, dtype: object

In [37]:
# guardo las provincias de las poblaciones que faltan
prov_para_pobl=df_soloartesanas[df_soloartesanas["Población"].isna()]["Provincia"].unique()
prov_para_pobl

array(['Valencia', 'Madrid', 'Valladolid', 'Navarra', 'Barcelona',
       'Cantabria', 'Guadalajara', 'Vizcaya', 'Girona', 'Salamanca'],
      dtype=object)

In [44]:
# ejemplo para comprobar
mask_prov_ej=(df_soloartesanas["Provincia"]=='Valencia') 
mask_sin_pobl_ej=mask_prov_ej & (df_soloartesanas["Población"].isna())
df_soloartesanas[mask_sin_pobl_ej]

,id,color,name,brand,description,image,Estilo de cerveza,Graduación alcohólica,Fermentación,Amargor (según la Unidad Internacional de Amargor),Población,Provincia,Comunidad Autónoma,Lúpulos,European Brewing Convention (EBC),Maltas,Temperatura de consumo,Grano,País
18,19,rubia,Birra & Blues Mago de Oz,(cervezas artesanas Birra & Blues),\nLa banda de rock Mago de Oz ya tiene su prop...,https://static1.soloartesanas.es/3014-large_de...,Blonde Ale,"4,5% Alc./Vol.",Ale,21 IBUs,NaN,Valencia,Valencia,NaN,7,NaN,NaN,NaN,NaN


In [45]:
prov_moda_ej=df_soloartesanas[mask_prov_ej]["Población"].mode()
prov_moda_ej

0    Massalfassar
dtype: object

In [46]:
# Para completar el dato de población, les voy a asignar por defecto
# la población que sea la moda en esa provincia


'''
    mask=(df_soloartesanas["color"]==df_estilo.loc[comb,"color"])&(df_soloartesanas["Fermentación"]==df_estilo.loc[comb,"Fermentación"])&(df_soloartesanas["Grano"]==df_estilo.loc[comb,"Grano"])
    mk_sin_estilo=mask & df_soloartesanas["Estilo"].isna()

    nuevo_estilo=df_soloartesanas[mask]["Estilo"].mode()
    
    df_soloartesanas.loc[mk_sin_estilo,"Estilo"]=nuevo_estilo[0]


'''

for provincia in prov_para_pobl:
    mask_prov=(df_soloartesanas["Provincia"]==provincia) 
    mask_sin_pobl=mask_prov & (df_soloartesanas["Población"].isna())
    prov_moda=df_soloartesanas[mask_prov]["Población"].mode()
    
    df_soloartesanas.loc[mask_sin_pobl,"Población"]=prov_moda[0]

# comprobamos
df_soloartesanas[df_soloartesanas["Población"].isna()]




,id,color,name,brand,description,image,Estilo de cerveza,Graduación alcohólica,Fermentación,Amargor (según la Unidad Internacional de Amargor),Población,Provincia,Comunidad Autónoma,Lúpulos,European Brewing Convention (EBC),Maltas,Temperatura de consumo,Grano,País


In [49]:
# ejemplo para comprobar
df_soloartesanas.loc[18,"Población"]

'Massalfassar'

In [ ]:
# Asigno "sin especificar" a las poblaciones que faltan pero sí conocemos su provincia y CC.AA
df_soloartesanas["Población"]=df_soloartesanas["Población"].fillna("sin especificar")

In [ ]:
df_soloartesanas.info()

In [ ]:
df_soloartesanas.columns

In [ ]:
# Unifico el campo "País" que tiene Nan y distintas grafías
df_soloartesanas["País"]="España"

# Muevo las columnas para que la información sobre la procedencia esté ordenada y al final del dataset
df_soloartesanas=df_soloartesanas[['id', 'color', 'name', 'brand', 'description', 'image',
       'Estilo de cerveza', 'Graduación alcohólica', 'Fermentación',
       'Amargor (según la Unidad Internacional de Amargor)', 'Lúpulos',
       'European Brewing Convention (EBC)', 'Maltas', 'Temperatura de consumo',
       'Grano', 'Población',
       'Provincia', 'Comunidad Autónoma', 'País']]


## Normalización de datos específicos sobre cervezas
A continuación se procede a revisar las columnas con información específica sobre las características de las cervezas, con el objetivo de que los datos sean uniformes y los tipos se correspondan con ellos. También se cambiarán los nombres de algunas columnas que resultan demasiado largos para trabajar.

In [ ]:
df_soloartesanas.columns

In [ ]:
'''
Como prueba. 
Hago una copia del df tal cual está en este punto para quitar todos los nulos 
y ver cómo se quedaría un a versión reducida pero completa
'''

df_sin_nulos=df_soloartesanas

In [ ]:
# cambio nombres de columnas demasiado largos
df_soloartesanas.rename(columns={'Estilo de cerveza':"Estilo",'Graduación alcohólica':"Graduación",'Amargor (según la Unidad Internacional de Amargor)':"IBU", },inplace=True)

In [ ]:
# Modifico la columna de porcentaje de alcohol (ahora es texto) para dejar graduación como float

# Valor ficticio(999) para evitar errores con los NAN
df_soloartesanas['Graduación']=df_soloartesanas['Graduación'].fillna("999")

# sustituir comas por puntos
df_soloartesanas['Graduación']=df_soloartesanas['Graduación'].apply(lambda x: x.replace(',','.'))

# extraer el dato numérico (nums antes y después de .)
df_soloartesanas['Graduación'] = df_soloartesanas["Graduación"].str.extract(r"(\d+(.\d+){0,1})", expand=True)[0]

# transformar en float
df_soloartesanas['Graduación']=df_soloartesanas['Graduación'].astype('float64')

In [ ]:
df_soloartesanas['Graduación'].unique()

In [ ]:
df_soloartesanas['Graduación'].value_counts()

In [ ]:
# ¿Cómo son las cervezas que no tienen dato de graducación?df_soloartesanas[df_soloartesanas['Graduación']==999.00]

In [ ]:
df_soloartesanas.loc[172,"description"]

In [ ]:
df_soloartesanas["IBU"].unique()

In [ ]:
df_soloartesanas["IBU"].value_counts()

In [ ]:
# limpio IBUS para dejar solo los números

# Valor ficticio(999) para evitar errores con los NAN
df_soloartesanas['IBU']=df_soloartesanas['IBU'].fillna("999")

# sustituir comas por puntos
df_soloartesanas['IBU']=df_soloartesanas['IBU'].apply(lambda x: x.replace(',','.'))

# extraer el dato numérico (nums antes y después de .)
df_soloartesanas['IBU']=df_soloartesanas['IBU'].str.extract(r"(\d+(.\d+){0,1})", expand=True)[0]

# transformar en float
df_soloartesanas['IBU']=df_soloartesanas['IBU'].fillna("999")
df_soloartesanas['IBU']=df_soloartesanas['IBU'].astype('float64')
df_soloartesanas['IBU'].unique()

In [ ]:
df_soloartesanas.info()

In [ ]:
df_soloartesanas['brand'].value_counts()

In [ ]:
# eliminar "cervezas artesanas " y los paréntesis en el campo "brand"
df_soloartesanas['brand']=df_soloartesanas['brand'].apply(lambda x: x.replace('(cervezas artesanas ',''))
df_soloartesanas['brand']=df_soloartesanas['brand'].apply(lambda x: x.replace(')',''))

In [ ]:
# ejemplo de una cerveza
df_soloartesanas.iloc[223]

In [ ]:
df_soloartesanas.columns

In [ ]:
df_soloartesanas["color"].unique()

In [ ]:
df_soloartesanas["name"].unique()

In [ ]:
df_soloartesanas["brand"].unique()

### Datos sobre el estilo de las cervezas
Uno de los datos más importantes en la valoración de las cervezas es su estilo. Encontramos que este campo tiene algunos registros sin especificar, por lo que buscamos la manera de completar la información basándonos en otros datos o, de manera puntual, buscando el estilo para unos casos concretos.


In [ ]:
df_soloartesanas["Estilo"].value_counts()

In [ ]:
df_soloartesanas["Estilo"].nunique()

In [ ]:
df_soloartesanas["Estilo"].unique()

In [ ]:
# ¿cómo son las que no tienen estilo?
df_estilo=df_soloartesanas[df_soloartesanas["Estilo"].isna()]

df_estilo=df_estilo.drop(columns=["id", "name","Graduación", "IBU","Lúpulos",
                                  "European Brewing Convention (EBC)","Maltas",
                                  'image',"brand","description",
                                 'Población', 'Provincia',
                                 'Comunidad Autónoma',
                                 'Temperatura de consumo', 'País'])
df_estilo

Observamos que para las 11 cervezas que no tienen clasificación de estilo podemos obtener el dato conociendo la fermentación, el color y el grano. 
Por tanto, prodecemos a obtener el estilo que más se repite en cada una de esas combinaciones y se lo añadimos a la cerveza. 

In [ ]:
# Eliminamos las combinaciones que se repiten
df_estilo.drop_duplicates(inplace=True)

#inicializamos el índice
df_estilo.reset_index(drop=True,inplace=True)
df_estilo

In [ ]:
# Buscamos el estilo que es la moda para cada combinación (ej.rubia & Ale & Cebada) y lo asignamos a cada cerveza sin estilo

combinaciones=range(df_estilo.columns.size+1)

for comb in combinaciones:
    
    mask=(df_soloartesanas["color"]==df_estilo.loc[comb,"color"])&(df_soloartesanas["Fermentación"]==df_estilo.loc[comb,"Fermentación"])&(df_soloartesanas["Grano"]==df_estilo.loc[comb,"Grano"])
    mk_sin_estilo=mask & df_soloartesanas["Estilo"].isna()

    nuevo_estilo=df_soloartesanas[mask]["Estilo"].mode()
    
    df_soloartesanas.loc[mk_sin_estilo,"Estilo"]=nuevo_estilo[0]

# comprobamos
df_soloartesanas[df_soloartesanas["Estilo"].isna()]


In [ ]:
# queda 1 cerveza sin estilo. Vemos su descripción para buscar más información
df_soloartesanas.loc[428,"description"]

In [ ]:
# Por la descripción encajaría en una Red IPA. Actualizamos el dato
df_soloartesanas.loc[428,"Estilo"]=' Red IPA'

In [ ]:
df_soloartesanas["Estilo"].unique()

In [ ]:
# hay varias cervezas del mismo tipo con distinta nomenclarura
#df_soloartesanas[df_soloartesanas["Estilo"].str.contains("India Pale Ale")]["Estilo"].unique()
df_soloartesanas[df_soloartesanas["Estilo"].str.contains("Pale Ale")]["Estilo"].unique()

In [ ]:
# Unificamos las India Pale Ale, excepto las que tienen otros calificativos, que son Valencia y Brown:
mask_pale_ale=df_soloartesanas["Estilo"].str.contains("Pale Ale")
mask_india=df_soloartesanas["Estilo"].str.contains("India")
mask_valencia=df_soloartesanas["Estilo"].str.contains("Valencia")
mask_brown=df_soloartesanas["Estilo"].str.contains("Brown")

# Asignamos ' India Pale Ale'
df_soloartesanas.loc[(mask_pale_ale & mask_india &~ mask_valencia &~ mask_brown ),["Estilo"]]=' India Pale Ale'

In [ ]:
# También hay que unificar las tipo "American Pale Ale - APA"
df_soloartesanas[df_soloartesanas["Estilo"].str.contains("America")]["Estilo"].value_counts()

In [ ]:
# máscaras
mask_america=df_soloartesanas["Estilo"].str.contains("America")

df_soloartesanas.loc[(mask_pale_ale & mask_america ),["Estilo"]]=' America Pale Ale'

In [ ]:
# cervezas Stout están bien diferenciadas
df_soloartesanas[df_soloartesanas["Estilo"].str.contains("Stout")]["Estilo"].value_counts()

In [ ]:
# las Witbier necesitan unificarse
df_soloartesanas[df_soloartesanas["Estilo"].str.contains("Wit")]["Estilo"].value_counts()

In [ ]:
df_soloartesanas.loc[df_soloartesanas["Estilo"].str.contains("Wit"),["Estilo"]]=' Witbier'

## La fermentación
Las cervezas se clasifican en cuanto a su fermentación como Ale (alta), Lager (baja) y espontánea, aunque en esta base de datos encontramos que las cervezas vienen clasificadas en tres tipos: Ale, Lager y Mixta.
Se ha detectado que hay 31 cervezas sin clasificar,pero que pueden ser identificadas por la información sobre su estilo. La mayoría de las cervezas tienen fermentación alta y entre ellas se encuentran las IPA, weizen, wit, stout y porter. Sin embargo, las de tipo rauchbier tienen fermentación baja.

In [ ]:
df_soloartesanas["Fermentación"].unique()

In [ ]:
print("Sin dato:",df_soloartesanas["Fermentación"].isna().sum())
df_soloartesanas["Fermentación"].value_counts()

In [ ]:
# Para las que no tienen dato de fermentación ¿cuál es su estilo?
#mask=df_soloartesanas["Fermentación"].isna()
df_soloartesanas.loc[(df_soloartesanas["Fermentación"].isna()),"Estilo"].value_counts()

In [ ]:
# Las cervezas que tienen estilo "Ale", "IPA","Weizen","Wit","Stout" y "Porter" son de fermentación Ale
# Las tipo Rauchbier son asigno Lager

# Como solamente hay 1 Rauchbier, actualizo esa con "Lager" 
df_soloartesanas.loc[(df_soloartesanas["Estilo"].str.contains("Rauchbier") & df_soloartesanas["Fermentación"].isna()),["Fermentación"]]='Lager'

# y asigno a las demás "Ale"
df_soloartesanas.loc[(df_soloartesanas["Fermentación"].isna()),["Fermentación"]]='Ale'

# compruebo
print("Sin dato:",df_soloartesanas["Fermentación"].isna().sum())
df_soloartesanas["Fermentación"].value_counts()

In [ ]:
df_soloartesanas["IBU"].unique()

In [ ]:
df_soloartesanas["Lúpulos"].unique()

In [ ]:
df_soloartesanas["European Brewing Convention (EBC)"].unique()

In [ ]:
df_soloartesanas["Maltas"].unique()

In [ ]:
df_soloartesanas["Temperatura de consumo"].unique()

### Unificamos el tipo de grano
A pesar de que este dato falta en bastantes registros, lo dejamos porque puede ser interesante para pintar. Para ello, primero unificamos los datos ya que algunos están escritos de varias formas.

In [ ]:
df_soloartesanas["Grano"].unique()

In [ ]:
df_soloartesanas["Grano"].value_counts()

In [ ]:
# Unifico cebada y trigo
granos=(df_soloartesanas["Grano"]=='Cebada y trigo') | (df_soloartesanas["Grano"]=='Trigo y cebada (50%)') | (df_soloartesanas["Grano"]=='Cebada y Trigo')
df_soloartesanas.loc[granos,"Grano"]='Cebada y trigo'


In [ ]:
df_soloartesanas.to_csv("data/soloartesanas_limpio.csv", sep = ';', index = False)

In [ ]:
df_soloartesanas.info()